In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, TimeDistributed, LSTM, Input, BatchNormalization, Conv2D, MaxPooling2D, Reshape, Conv1D, GlobalAveragePooling1D, MaxPooling1D, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.losses import categorical_crossentropy
import tensorflow_hub as hub
from PIL import Image
import gzip
from keras.preprocessing.sequence import pad_sequences
import spacy
import h5py
import os
import cv2
import pickle
import re
import shutil
import glob
import gzip
%matplotlib inline

In [ ]:
import gzip

In [ ]:
path_gloss_files = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.train.annotations_only.gzip'
path_dev_gloss = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.dev.annotations_only.gzip'
path_test_gloss = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.test.annotations_only.gzip'

# reading gloss files

In [ ]:
with gzip.open(path_gloss_files,'rb') as f:
  annotation_gloss = pickle.load(f)

In [ ]:
with gzip.open(path_dev_gloss,'rb') as f:
  annotation_dev = pickle.load(f)

In [ ]:
with gzip.open(path_test_gloss,'rb')as f :
  annotation_test = pickle.load(f)

In [ ]:
annotation_gloss = pd.DataFrame(annotation_gloss)

In [ ]:
annotation_gloss_dev = pd.DataFrame(annotation_dev)

In [ ]:
annotation_gloss_test = pd.DataFrame(annotation_test)

# preprocessing glosses and texts

In [ ]:
for i in range(len(annotation_gloss)):
  annotation_gloss.iloc[i,2] = 'startseq '+annotation_gloss.iloc[i,2].lower() + ' endseq'
  annotation_gloss.iloc[i,3] = 'startseq '+annotation_gloss.iloc[i,3].lower() + ' endseq'
  annotation_gloss.iloc[i,3] = annotation_gloss.iloc[i,3].replace(' .','')
  #annotation_gloss.iloc[i,3] =annotation_gloss.iloc[i,3][-2].replace(' ','')
 

In [ ]:
for i in range(len(annotation_gloss_dev)):
  annotation_gloss_dev.iloc[i,2] = 'startseq '+annotation_gloss_dev.iloc[i,2].lower() + ' endseq'
  annotation_gloss_dev.iloc[i,3] = 'startseq '+annotation_gloss_dev.iloc[i,3].lower() + ' endseq'
  annotation_gloss_dev.iloc[i,3] = annotation_gloss_dev.iloc[i,3].replace(' .','')

In [ ]:
for i in range(len(annotation_gloss_test)):
  annotation_gloss_test.iloc[i,2] = 'startseq '+annotation_gloss_test.iloc[i,2].lower() + ' endseq'
  annotation_gloss_test.iloc[i,3] = 'startseq '+annotation_gloss_test.iloc[i,3].lower() + ' endseq'
  annotation_gloss_test.iloc[i,3] = annotation_gloss_test.iloc[i,3].replace(' .','')

In [ ]:
annotation_gloss.head()

,name,signer,gloss,text
0,train/11August_2010_Wednesday_tagesschau-1,Signer08,startseq jetzt wetter morgen donnerstag zwoelf...,startseq und nun die wettervorhersage für morg...
1,train/11August_2010_Wednesday_tagesschau-4,Signer08,startseq ort regen durch regen koennen uebersc...,startseq mancherorts regnet es auch länger und...
2,train/11August_2010_Wednesday_tagesschau-5,Signer08,startseq nordwest heute nacht trocken bleiben ...,startseq im nordwesten bleibt es heute nacht m...
3,train/11August_2010_Wednesday_tagesschau-6,Signer08,startseq tagsueber oft regen gewitter koennen ...,startseq auch am tag gibt es verbreitet zum te...
4,train/11August_2010_Wednesday_tagesschau-7,Signer08,startseq wolke loch speziell nordwest endseq,startseq größere wolkenlücken finden sich vor ...


In [ ]:
annotation_gloss_dev.head()

,name,signer,gloss,text
0,dev/11August_2010_Wednesday_tagesschau-2,Signer08,startseq druck tief kommen endseq,startseq tiefer luftdruck bestimmt in den näch...
1,dev/11August_2010_Wednesday_tagesschau-3,Signer08,startseq es-bedeutet viel wolke und koennen re...,startseq das bedeutet viele wolken und immer w...
2,dev/11August_2010_Wednesday_tagesschau-8,Signer08,startseq wind maessig schwach region wenn gewi...,startseq meist weht nur ein schwacher wind aus...
3,dev/25October_2010_Monday_tagesschau-22,Signer01,startseq mittwoch regen koennen nordwest wahrs...,startseq am mittwoch hier und da nieselregen i...
4,dev/05May_2011_Thursday_tagesschau-25,Signer08,startseq jetzt wetter wie-aussehen morgen frei...,startseq und nun die wettervorhersage für morg...


In [ ]:
annotation_gloss_test.head()

,name,signer,gloss,text
0,test/25October_2010_Monday_tagesschau-17,Signer01,startseq regen schnee region verschwinden nord...,startseq regen und schnee lassen an den alpen ...
1,test/25October_2010_Monday_tagesschau-24,Signer01,startseq donnerstag nordwest regen region sonn...,startseq am donnerstag regen in der nordhälfte...
2,test/15December_2010_Wednesday_tagesschau-37,Signer05,startseq kraeftig ab morgen frueh meistens sch...,startseq vom nordmeer zieht ein kräftiges tief...
3,test/10March_2011_Thursday_heute-58,Signer01,startseq wochenende sonne samstag schoen tempe...,startseq sonnig geht es auch ins wochenende sa...
4,test/14August_2009_Friday_tagesschau-62,Signer05,startseq deutsch land morgen hoch druck kommen...,startseq deutschland liegt morgen unter hochdr...


# calculation max length in texts

In [ ]:
gloss_gloss = []
for i in range(7096):
  gloss_gloss.append(annotation_gloss.iloc[i,3])
for i in range(len(annotation_gloss_dev)):
  gloss_gloss.append(annotation_gloss_dev.iloc[i,3])
for i in range(len(annotation_gloss_test)):
  gloss_gloss.append(annotation_gloss_test.iloc[i,3])

In [ ]:
len(gloss_gloss)

8257

In [ ]:
gloss_gloss[0]

'startseq und nun die wettervorhersage für morgen donnerstag den zwölften august endseq'

In [ ]:
length_check = []
for i in range (len(gloss_gloss)):
  length_check.append(gloss_gloss[i].split(' '))
  


In [ ]:
len(length_check)

8257

In [ ]:
# removeing all  single length characters
for i in range(len(length_check)):
  for j in length_check[i]:
    if(len(j)<=1):
      length_check[i].remove(j)

In [ ]:
count=0
for i in range(len(length_check)):
  if(len(length_check[i])>count):
    count = len(length_check[i])

In [ ]:
count

54

In [ ]:
length_check[0]

['startseq',
 'und',
 'nun',
 'die',
 'wettervorhersage',
 'für',
 'morgen',
 'donnerstag',
 'den',
 'zwölften',
 'august',
 'endseq']

# max length is 54

# creating a tokenizer

In [ ]:
#creating a list of all unique words

In [ ]:
unique_list_text = []
for i in range(len(length_check)):
  for j in length_check[i]:
    if (j not in unique_list_text):
      unique_list_text.append(j)

In [ ]:
len(unique_list_text)

3001

# removing words with frequency less than 10 times

In [ ]:
dictionary_unq = {}

In [ ]:
for i in range(len(unique_list_text)):
  count= 0
  for j in length_check:
    for k in j:
      if(unique_list_text[i] == k):
        count+=1
  dictionary_unq[unique_list_text[i]] = count


In [ ]:
for keys in dictionary_unq.keys():
  if dictionary_unq[keys]<5:
    unique_list_text.remove(keys)

In [ ]:
unique_list_text.append('<UNK>')

# creating a tokenizer

In [ ]:
def create_tokenizer(unique_list_text):
  tokenizer = tf.keras.preprocessing.text.Tokenizer()
  tokenizer.fit_on_texts(unique_list_text)
  return tokenizer


In [ ]:
tokenizer = create_tokenizer(unique_list_text)

In [ ]:
vocab_size_text = len(tokenizer.word_index)+1

In [ ]:
print("vocalb size",vocab_size_text)

vocalb size 1202


# creating dataset for tokenizer

In [ ]:
gloss_token = []
text_token = []
for i in range(len(annotation_gloss)):
  gloss_token.append(annotation_gloss.iloc[i,2])
  text_token.append(annotation_gloss.iloc[i,3])

In [ ]:
text_token[0]

'startseq und nun die wettervorhersage für morgen donnerstag den zwölften august endseq'

In [ ]:
gloss_token_dev = []
text_token_dev = []
for i in range(len(annotation_gloss_dev)):
  gloss_token_dev.append(annotation_gloss_dev.iloc[i,2])
  text_token_dev.append(annotation_gloss_dev.iloc[i,3])

In [ ]:
gloss_token_test = []
text_token_test = []
for i in range(len(annotation_gloss_test)):
  gloss_token_test.append(annotation_gloss_test.iloc[i,2])
  text_token_test.append(annotation_gloss_test.iloc[i,3])

In [ ]:
len(gloss_token)

7096

In [ ]:
len(gloss_token_dev)

519

In [ ]:
len(gloss_token_test)

642

# tokenizing whole glosses

In [ ]:
gloss_token[0]

'startseq jetzt wetter morgen donnerstag zwoelf februar endseq'

In [ ]:

def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

In [ ]:
X_train = encode_sequences(tokenizer,35,gloss_token)

In [ ]:
X_train[0]

array([  1, 552, 118,   7,   8, 380,  12,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
np.shape(X_train)


(7096, 35)

In [ ]:
X_dev = encode_sequences(tokenizer,35,gloss_token_dev)

In [ ]:
X_test = encode_sequences(tokenizer,35,gloss_token_test)

In [ ]:
np.shape(X_dev)

(519, 35)

In [ ]:
def encode_output(sequences, vocab_size):
  ylist = list()
  for sequence in sequences:
    encoded = to_categorical(sequence, num_classes=vocab_size)
    ylist.append(encoded)
  y = np.array(ylist)
  print("shape of sequences {}".format(np.shape(sequences)))
  y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
  return y


In [ ]:
Y_output_token = encode_sequences(tokenizer,54,text_token)

In [ ]:
Y_output = encode_output(Y_output_token,1202)

shape of sequences (7096, 54)


In [ ]:
Y_dev_token = encode_sequences(tokenizer,54,text_token_dev)

In [ ]:
Y_dev = encode_output(Y_dev_token,1202)

shape of sequences (519, 54)


In [ ]:
Y_test_token = encode_sequences(tokenizer,54,text_token_test)

In [ ]:
Y_test = encode_output(Y_test_token,1202)

shape of sequences (642, 54)


# creating a model to convert gloss to text

In [ ]:
import collections

import helper
import numpy as np
#import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers.merge import add

In [ ]:
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

# define model
model = define_model(vocab_size_text, vocab_size_text, 35, 54, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
# summarize defined model
print(model.summary())
#plot_model(model, to_file='model.png', show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 35, 256)           307712    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 54, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 54, 256)           525312    
_________________________________________________________________
time_distributed (TimeDistri (None, 54, 1202)          308914    
Total params: 1,667,250
Trainable params: 1,667,250
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train, Y_output, epochs=100, batch_size=64, validation_data=(X_dev, Y_dev), verbose=2)

Epoch 1/100
111/111 - 133s - loss: 2.2929 - accuracy: 0.7163 - val_loss: 1.5824 - val_accuracy: 0.7443
Epoch 2/100
111/111 - 106s - loss: 1.6154 - accuracy: 0.7337 - val_loss: 1.4996 - val_accuracy: 0.7489
Epoch 3/100
111/111 - 104s - loss: 1.5810 - accuracy: 0.7379 - val_loss: 1.4806 - val_accuracy: 0.7512
Epoch 4/100
111/111 - 105s - loss: 1.5613 - accuracy: 0.7401 - val_loss: 1.4524 - val_accuracy: 0.7548
Epoch 5/100
111/111 - 105s - loss: 1.5315 - accuracy: 0.7428 - val_loss: 1.4298 - val_accuracy: 0.7578
Epoch 6/100
111/111 - 104s - loss: 1.5042 - accuracy: 0.7447 - val_loss: 1.3972 - val_accuracy: 0.7593
Epoch 7/100
111/111 - 104s - loss: 1.4736 - accuracy: 0.7469 - val_loss: 1.3674 - val_accuracy: 0.7628
Epoch 8/100
111/111 - 105s - loss: 1.4528 - accuracy: 0.7481 - val_loss: 1.3559 - val_accuracy: 0.7627
Epoch 9/100
111/111 - 104s - loss: 1.4238 - accuracy: 0.7500 - val_loss: 1.3293 - val_accuracy: 0.7645
Epoch 10/100
111/111 - 104s - loss: 1.4022 - accuracy: 0.7508 - val_loss:

KeyboardInterrupt: ignored

# prediction 

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [ ]:
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [np.argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [ ]:
#!pip install jiwer

     |████████████████████████████████| 51kB 2.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149805 sha256=48d2bd73acbc31f7f3ededd64b7f9a6b26e68dcc0df94ebb7ac1d8c207a31ed1
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
import jiwer

In [ ]:
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset,raw_result):
  actual, predicted = list(), list()
  actual_temp, predicted_temp = list(),list()
  for i, source in enumerate(sources):
  # translate encoded source text
    source = source.reshape((1, source.shape[0]))
    translation = predict_sequence(model, tokenizer, source)
    raw_target, raw_src = raw_result[i],raw_dataset[i]
    if i < 10:
      print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
      actual.append([raw_target.split()])
      predicted.append(translation.split())
  # for i in range(len(sources)):
  #   temp_sent = ''
  #   #temp_desc = ''
  #   print("predicted i {},prediction{} ".format(i,predicted[i]))
  #   for j in predicted[i]:
  #     print("word in predicted i is ",j)
  #     temp_sent = temp_sent+" "+j
  #   #temp_desc = temp_desc + description[i]
  #   predicted_temp.append(temp_sent)
  #   #actual_temp.append()
    
  # for i in range(len(sources)):
  #   temp_desc = ''
  #   print("actual i {},actual{} ".format(i,sources[i]))
  #   for j in descriptions[i]:
  #     temp_desc = temp_desc+" "+j

  #   actual_temp.append(temp_desc)


  #calculate BLEU score
  print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
  print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
  print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
  print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
  #print("word error rate is",jiwer.wer(raw_src,translation))

In [ ]:
evaluate_model(model, tokenizer, X_test, gloss_token_test,text_token_test)

src=[startseq regen schnee region verschwinden nord regen koennen region stern koennen sehen endseq], target=[startseq regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar endseq], predicted=[startseq im der regnet es es der der der der der der der der]
src=[startseq donnerstag nordwest regen region sonne wolke wechselhaft dann freitag aehnlich wetter endseq], target=[startseq am donnerstag regen in der nordhälfte in der südhälfte mal sonne mal wolken ähnliches wetter dann auch am freitag endseq], predicted=[startseq am donnerstag wird es im im regen regen und und und und und]
src=[startseq kraeftig ab morgen frueh meistens schnee schneien kalt regen endseq], target=[startseq vom nordmeer zieht ein kräftiges tief heran und bringt uns ab den morgenstunden heftige schneefälle zum teil auch gefrierenden regen endseq], predicted=[startseq morgen morgen es es es regen regen regen regen regen]
src=[startseq wochenende so

# creating attention models for learning gloss to german langauge


### creating index to word and word to index mapping